<a href="https://colab.research.google.com/github/Rafleon00/03MAIR--Algoritmos-de-Optimizacion--2019/blob/master/AG3/AG3_Mod_Ruben_Alvarez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG - Modificación Actividad Guiada 3<br>

Rubén Álvarez Fernández<br>

https://github.com/Rafleon00/03MAIR--Algoritmos-de-Optimizacion--2019/tree/master/AG3<br>

In [0]:
# Problema TSP
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f1189a8f4a8>)

In [0]:
!pip install tsplib95

    100% |████████████████████████████████| 1.6MB 17.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
Successfully built networkx
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
import tsplib95
import random
from math import e


problem = tsplib95.load_problem(file)


#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

In [0]:
#Devuelve el factorial de un numero
def factorial(n):
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
  

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
  return solucion


#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)
                            

#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


In [62]:
# Busqueda aleatoria TSP (SIN MODIFICACIONES)
def busqueda_aleatoria(problem, N): # N:Numero de iteraciones 
  
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  mejor_distancia = 1e100
  
  for i in range(N):
    solucion = crear_solucion(Nodos)
    distancia = distancia_total(solucion, problem)
    
    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia
    
  print ('Mejor solucion' , mejor_solucion)
  print ('Distancia' , mejor_distancia) 
  return mejor_solucion
  
sol_busqueda_aleatoria = busqueda_aleatoria(problem, 10000)

Mejor solucion [0, 29, 21, 41, 30, 20, 9, 25, 8, 34, 23, 2, 26, 4, 1, 14, 37, 32, 38, 39, 24, 18, 12, 15, 36, 35, 31, 17, 7, 11, 10, 28, 27, 33, 13, 19, 22, 40, 6, 16, 3, 5]
Distancia 3552


In [0]:
# Generador de vecindad
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

In [64]:
# Busqueda local TSP (SIN MODIFICACIONES)
def busqueda_local(problem, N): # N:Numero de iteraciones 
  
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  mejor_distancia = 1e100
  
  solucion_referencia = crear_solucion(Nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
    
    solucion_referencia = vecina
  
  print ('Mejor solucion' , mejor_solucion)
  print ('Distancia' , mejor_distancia) 
  return mejor_solucion
  
  
sol_busqueda_local = busqueda_local(problem, 100) 

Mejor solucion [0, 30, 39, 22, 38, 32, 7, 17, 31, 20, 34, 33, 35, 36, 37, 15, 16, 14, 19, 1, 4, 6, 5, 13, 26, 18, 12, 11, 25, 10, 3, 2, 27, 28, 29, 9, 21, 24, 40, 23, 41, 8]
Distancia 1707


In [0]:
# Generación de una vecina aleatoria (no mejora función anterior de vecinos)
def genera_vecina_aleatorio(solucion): 
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  
  vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  
  return vecina

# Función de probabilidad
def probabilidad(T,d):
  r=random.random()
  if(r <= (e**(-1*d)/(T*1.0))):
    return True
  else:
    return False
  
def bajar_temperatura(T):
  return T-1

In [66]:
# Recocido simulado TSP (SIN MODIFICACIONES)
def recocido_simulado(problem, TEMPERATURA):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  while TEMPERATURA > 0:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    TEMPERATURA = bajar_temperatura(TEMPERATURA)
  
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol = recocido_simulado(problem, 100)

La mejor solución encontrada es [0, 2, 28, 8, 29, 30, 32, 31, 35, 36, 17, 19, 13, 18, 10, 41, 23, 40, 24, 21, 14, 16, 15, 37, 7, 1, 3, 4, 27, 20, 33, 34, 38, 22, 39, 9, 25, 11, 12, 26, 5, 6]
con una distancia total de 1838


In [0]:
# H: Hormiga (Recorrido parcial TSP)

# Add_Nodo Modificación: Función de probabilidad para selección de nodos, escogiendo 
# el que presente mayor probabilidad en cada ocasión

def Add_Nodo_Prob(problem, H ,T, alfa, beta) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  Nodos_posibles = list(set(range(1,len(Nodos))) - set(H)) 
  f_prob = []
  denom = 0
  for i in range(len(Nodos_posibles)):
    dist = distancia(H[-1],Nodos_posibles[i], problem)
    ferom = T[H[-1]][Nodos_posibles[i]]
    num = ferom**alfa + (1/dist)**beta
    denom = num + denom
    f_prob.append(num)
  prob = [(f_prob[i]/denom) for i in range(len(f_prob))]
  sel = prob.index(max(prob))
                     
  return Nodos_posibles[sel]


def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

# Evaporar_Feromonas Modificación: Revisión del bucle anidado para que 
# realice correctamente la evaporación

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max((T[i][j] - 0.3) , 1) for j in range(len(Nodos)) ] for i in range(len(Nodos))]
  return T

In [89]:
# Colonia de Hormigas TSP
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges()) 
  
  # Coeficientes función probabilidad para creación de nodos
  alfa = 0.5
  beta = 0.5
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo_Prob(problem, Hormiga[h] , T, alfa, beta)
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )

    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    
    
  #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia = distancia_actual
  
  print ('Mejor ruta')
  print(mejor_solucion)
  print ('\n')
  print ('Distancia')
  print(mejor_distancia)  
  
hormigas(problem, 100)


Mejor ruta
[0, 1, 6, 4, 3, 2, 27, 28, 29, 30, 32, 34, 20, 33, 31, 17, 37, 15, 14, 16, 19, 13, 5, 26, 18, 12, 11, 25, 10, 8, 9, 23, 41, 21, 39, 22, 38, 24, 40, 7, 36, 35]


Distancia
1630
